In [ ]:
import os
import warnings
import pickle    # chunk, vectorDB 저장한것 사용
from dotenv import load_dotenv

# 경고메세지 삭제
warnings.filterwarnings('ignore')
load_dotenv()

# openapi key 확인
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError('.env확인,  key없음')

# 필수 라이브러리 로드
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time

class SimpleRAGSystem:
    '''간단한 RAG 시스템 래퍼 클래스'''
    def __init__(self, vectorstore, llm, retriever_k=3):
        self.vectorstore = vectorstore
        self.llm = llm
        self.retriever = vectorstore.as_retriever(seaarch_type = 'similarity', search_kwargs={'k':retriever_k})
        # self.retriever_chain = self._retriever_basic_chain()
        self.chain = self._build_chain()
    

    
    
    # def _retriever_basic_chain(self): # -------------> 내부 문서 찾는 프롬프트 수정
    #     '''retriever 검색'''
    #     basic_prompt = ChatPromptTemplate.from_messages([
    #             ("system", """당신은 제공된 문맥(Context)을 바탕으로 질문에 답변하는 AI 어시스턴트입니다.

    #         규칙:
    #         1. 제공된 문맥 내의 정보만을 사용하여 답변하세요.
    #         2. 문맥에 없는 정보는 "제공된 문서에서 해당 정보를 찾을 수 없습니다."라고 답하세요.
    #         3. 답변은 한국어로 명확하고 간결하게 작성하세요.
    #         4. 가능하면 구조화된 형태(목록, 번호 등)로 답변하세요."""),
    #             ("human", """문맥(Context): {context}

    #         질문: {question}

    #         답변:""")
    #         ])

    #     return (
            
    #     )


    def _build_chain(self): ### ---------> 최종 사용자에게 전달되는 프롬프트 수정
        '''RAG 체인 구성''' 
        
        prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a professional AI research assistant specializing in HuggingFace Daily Papers.

        # Role
        Maintain context across multiple turns of conversation while answering based on retrieved papers.

        ## Conversational Guidelines 
        1. **Context Awareness**: Reference previous messages when relevant.

        2. **Consistency**: Maintain consistent terminology across the conversation.

        3. **Building Upon**: Build upon earlier answers with new retrieved information.

        4. **Clarification**: If asked for clarification, refer back to previously cited papers.

        ## Answer Rules
        1. **Source-based**: Only use information from the provided context.

        2. **Accuracy**: Do not make up information.

        3. **Language**: Answer in Korean.

        ## When Unable to Answer
        If you cannot find the information:
        "사용자가 질문한 사항은 허깅페이스의 최근 5주차 데이터에 없습니다. GPT-4o 모델로 검색하여 답변하겠습니다."
        라고 답변하고, 너가 검색해서 300자 내로 대답해. 이때, 사용자가 질문한 언어로 대답해줘."""),
                
                ("human", """## Previous Conversation 
        {chat_history}

        ## Retrieved Papers 
        {context}

        ## Current Question 
        {question}

        ## Answer """)
            ])
        return(
            {
            'context': self.retriever | self._format_docs,
            'question': RunnablePassthrough(),
            'chat_history': lambda x: ""
            }
            | prompt
            | self.llm
            | StrOutputParser()
        )
    
    
    @staticmethod   
    def _format_docs(docs):
        return '\n\n'.join([doc.page_content for doc in docs])
    

    def ask(self, question:str) -> str:
        '''질문에 답변'''
        return self.chain.invoke(question)
    

    def ask_with_sources(self, question:str) -> dict : 
        '''질문에 답변 + 출처 반환'''
        answer = self.chain.invoke(question)
        sources = self.retriever.invoke(question)
        return {
            'answer' : answer
            # 'source' : [ doc.metadata.get('source', 'unknown') for doc in sources]
        }
    


if __name__ == '__main__' :
    # vectorDB 로드 (vectorstore)
    
    pkl_path = r"C:\00project\SKN20-3rd-2TEAM\01_data\chunks\chunks_all.pkl"
    
    with open(pkl_path, 'rb') as f:
        chunks = pickle.load(f)

    # ---- metadata 제거 ----
    for doc in chunks:
        doc.metadata = {}

    vectorstore = Chroma.from_documents(
        documents=chunks,
        collection_name='test',
        embedding=OpenAIEmbeddings(model='text-embedding-3-small')
    )

  
    llm = ChatOpenAI( model = 'gpt-4o-mini', temperature=0 )

    rag_system = SimpleRAGSystem(vectorstore, llm)

    print('래퍼 클래스 테스트: ')
    result = rag_system.ask_with_sources("VectorDB의 종류를 알려주세요")
    print(f'질문: VectorDB의 종류를 알려주세요')
    print(f"답변: {result['answer']}")  
    # print(f"출처: {result['source']}")

래퍼 클래스 테스트: 
질문: VectorDB의 종류를 알려주세요
답변: 사용자가 질문한 사항은 허깅페이스의 최근 5주차 데이터에 없습니다. GPT-4o 모델로 검색하여 답변하겠습니다. 

VectorDB의 종류에는 여러 가지가 있으며, 대표적으로는 FAISS, Annoy, Milvus, Weaviate, Pinecone 등이 있습니다. 이들 각각은 데이터 검색 및 유사도 검색을 위한 다양한 기능과 최적화된 성능을 제공합니다.


In [14]:
import os
import warnings
import pickle    # chunk, vectorDB 저장한것 사용
from dotenv import load_dotenv

# 경고메세지 삭제
warnings.filterwarnings('ignore')
load_dotenv()

# openapi key 확인
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError('.env확인,  key없음')

# 필수 라이브러리 로드
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time

class SimpleRAGSystem:
    '''간단한 RAG 시스템 래퍼 클래스'''
    def __init__(self, vectorstore, llm, retriever_k=3):
        self.vectorstore = vectorstore
        self.llm = llm
        self.retriever = vectorstore.as_retriever(seaarch_type = 'similarity', search_kwargs={'k':retriever_k})
        # self.retriever_chain = self._retriever_basic_chain()
        self.chain = self._build_chain()
    

    
    
    # def _retriever_basic_chain(self): # -------------> 내부 문서 찾는 프롬프트 수정
    #     '''retriever 검색'''
    #     basic_prompt = ChatPromptTemplate.from_messages([
    #             ("system", """당신은 제공된 문맥(Context)을 바탕으로 질문에 답변하는 AI 어시스턴트입니다.

    #         규칙:
    #         1. 제공된 문맥 내의 정보만을 사용하여 답변하세요.
    #         2. 문맥에 없는 정보는 "제공된 문서에서 해당 정보를 찾을 수 없습니다."라고 답하세요.
    #         3. 답변은 한국어로 명확하고 간결하게 작성하세요.
    #         4. 가능하면 구조화된 형태(목록, 번호 등)로 답변하세요."""),
    #             ("human", """문맥(Context): {context}

    #         질문: {question}

    #         답변:""")
    #         ])

    #     return (
            
    #     )


    def _build_chain(self): ### ---------> 최종 사용자에게 전달되는 프롬프트 수정
        '''RAG 체인 구성''' 
        
        prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a professional AI research assistant specializing in HuggingFace Daily Papers.

        # Role
        Maintain context across multiple turns of conversation while answering based on retrieved papers.

        ## Conversational Guidelines 
        1. **Context Awareness**: Reference previous messages when relevant.

        2. **Consistency**: Maintain consistent terminology across the conversation.

        3. **Building Upon**: Build upon earlier answers with new retrieved information.

        4. **Clarification**: If asked for clarification, refer back to previously cited papers.

        ## Answer Rules
        1. **Source-based**: Only use information from the provided context.

        2. **Accuracy**: Do not make up information.

        3. **Language**: Answer in Korean.

        ## When Unable to Answer
        If you cannot find the information:
        "제공된 논문에서 해당 정보를 찾을 수 없습니다. 다른 질문을 해주시거나, 더 구체적으로 질문해주세요."
라고 답변합니다."""),
                
                ("human", """## Previous Conversation 
        {chat_history}

        ## Retrieved Papers 
        {context}

        ## Current Question 
        {question}

        ## Answer """)
            ])
        return(
            {
            'context': self.retriever | self._format_docs,
            'question': RunnablePassthrough(),
            'chat_history': lambda x: ""
            }
            | prompt
            | self.llm
            | StrOutputParser()
        )
    
    
    @staticmethod   
    def _format_docs(docs):
        return '\n\n'.join([doc.page_content for doc in docs])
    

    def ask(self, question:str) -> str:
        '''질문에 답변'''
        return self.chain.invoke(question)
    

    def ask_with_sources(self, question:str) -> dict : 
        '''질문에 답변 + 출처 반환'''
        answer = self.chain.invoke(question)
        sources = self.retriever.invoke(question)
        return {
            'answer' : answer
            # 'source' : [ doc.metadata.get('source', 'unknown') for doc in sources]
        }
    


if __name__ == '__main__' :
    # vectorDB 로드 (vectorstore)
    
    pkl_path = r"C:\00project\SKN20-3rd-2TEAM\01_data\chunks\chunks_all.pkl"
    
    with open(pkl_path, 'rb') as f:
        chunks = pickle.load(f)

    # ---- metadata 제거 ----
    for doc in chunks:
        doc.metadata = {}

    vectorstore = Chroma.from_documents(
        documents=chunks,
        collection_name='test',
        embedding=OpenAIEmbeddings(model='text-embedding-3-small')
    )

  
    llm = ChatOpenAI( model = 'gpt-4o-mini', temperature=0 )

    rag_system = SimpleRAGSystem(vectorstore, llm)

    print('래퍼 클래스 테스트: ')
    result = rag_system.ask_with_sources("VectorDB의 종류를 알려주세요")
    print(f'질문: VectorDB의 종류를 알려주세요')
    print(f"답변: {result['answer']}")  
    # print(f"출처: {result['source']}")

래퍼 클래스 테스트: 
질문: VectorDB의 종류를 알려주세요
답변: 제공된 논문에서 해당 정보를 찾을 수 없습니다. 다른 질문을 해주시거나, 더 구체적으로 질문해주세요.


In [ ]:
from langchain.prompts import ChatPromptTemplate
    



prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a professional AI research assistant specializing in HuggingFace Daily Papers.


## Role (역할)
Maintain context across multiple turns of conversation while answering based on retrieved papers.
여러 차례의 대화에 걸쳐 맥락을 유지하면서 검색된 논문을 기반으로 답변합니다.

## Conversational Guidelines (대화 가이드라인)
1. **Context Awareness (맥락 인식)**: Reference previous messages when relevant.
   관련성이 있을 때 이전 메시지를 참조합니다.

2. **Consistency (일관성)**: Maintain consistent terminology across the conversation.
   대화 전반에 걸쳐 일관된 용어를 유지합니다.

3. **Building Upon (보완)**: Build upon earlier answers with new retrieved information.
   새로 검색된 정보로 이전 답변을 보완합니다.

4. **Clarification (명확화)**: If asked for clarification, refer back to previously cited papers.
   명확화를 요청받으면 이전에 인용한 논문을 다시 참조합니다.

## Answer Rules (답변 규칙)
1. **Source-based (출처 기반)**: Only use information from the provided context.
   반드시 제공된 문맥의 정보만 사용합니다.

2. **Accuracy (정확성)**: Do not make up information.
   정보를 지어내지 않습니다.

3. **Language (언어)**: Answer in Korean.
   한국어로 답변합니다.

## When Unable to Answer (답변 불가 시)
If you cannot find the information:
정보를 찾을 수 없으면:
"제공된 논문에서 해당 정보를 찾을 수 없습니다. 다른 질문을 해주시거나, 더 구체적으로 질문해주세요."
라고 답변합니다."""),
        
        ("human", """## Previous Conversation (이전 대화)
{chat_history}

## Retrieved Papers (참조 논문)
{context}

## Current Question (현재 질문)
{question}

## Answer (답변)""")
    ])